In [ ]:
import pickle
import pandas as pd
import os
import datetime
from nltk.corpus import stopwords
from multiprocessing import Pool
import telegram_send as ts

In [ ]:
# define number of cores to be used for multi core machines

In [ ]:
nc=int(input('How many cores should be used?: '))

In [ ]:
# load the list of german place names and english and german stopwords

In [ ]:
with open('/path/to/folder/staedte_osm.txt') as f:
    ct=f.readlines()

In [ ]:
ct=[x.strip().lower()[1:-1] for x in ct]

In [ ]:
sw=stopwords.words('english')

In [ ]:
swd=stopwords.words('german')

In [ ]:
# combine lists of place names and stopwords

In [ ]:
stopwords=sw+swd+ct

In [ ]:
# replace german specific characters in the stopwords and place names

In [ ]:
stopwords=[x.replace('ä','ae').replace('ö','oe').replace('ü','ue').replace('ß','ss') for x in stopwords]

In [ ]:
# create a list of file paths for the text data

In [ ]:
ppath='/path/to/dataframes/containing/text'

In [ ]:
pfiles=[os.path.join(ppath,x) for x in os.listdir(ppath)]

In [ ]:
pfiles.sort()

In [ ]:
# define the output folder filepath and the functions for removing
# the stopwords and words containing non-ASCII characters

In [ ]:
def remove_stopwords(cell):
    if pd.notna(cell):
        return ' '.join([wrd for wrd in cell.split(' ') if wrd not in stopwords and len(wrd) > 2 and wrd.isascii()])

In [ ]:
pout='/path/to/output/folder'

In [ ]:
def rms(pfile):
    
    print('*'*10)
    print('  ',pfile,'\n',datetime.datetime.now())
    print('*'*10)

    start_count=datetime.datetime.now()
    
    npf=os.path.join(pout,os.path.basename(pfile))
    
    if os.path.basename(pfile) in os.listdir(pout):
        print('Stopwords already removed')
        
    else:
        with open (pfile ,'rb') as f:
            df=pickle.load(f)
        
        print('Removing stopwords...')
        df2=df.applymap(remove_stopwords) # apply function to text data
        with open (npf ,'wb') as f:
            pickle.dump(df2,f)
    
    end_count=datetime.datetime.now()
    tt=end_count-start_count
    
    print(f'Time taken for {os.path.basename(pfile)} was {tt} \n')

In [ ]:
# try and apply the functions to the files containing the text and 
# send notice of the success or failure of the operation to a telegram account

In [ ]:
try:
    with Pool(nc) as p:
        p.map(rms, pfiles)
except:
    ts.send(messages=['error'])

In [ ]:
ts.send(messages=['finished removing stopwords'])